# Pytorch Model 

this notebook will demonstrate the same techniques and practices used in Emotion_Classifier notebook but using Pytorch instead of Tensorflow. Pytorch comes with enough tools that we don't need Librosa as well and Pytorch's audio module "Torchaudio" has audio tranformers that are based on Librosa's fundementals. Then biggest difference might be that Pytorch will turn an audiofile into a Tensor instead of a Numpy array like with Librosa. 

In [88]:
# Importing Necessary Libraries 

# Standard Python Operations 
import pandas as pd 
import numpy as np
import math
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Audio and torch data
from scipy import signal 
import torchaudio 
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(0)


# Neural Network 
import torch 
import torch.nn as nn 
import warnings
warnings.filterwarnings("ignore")


# Preparing data 
First the data will be prepared the same way as in my other notebooks. A Pandas Dataframe will be created that contains the Path of the audio file and its respected class label for each observation. Then I will create a custom Data Class object using Pytorch's Data Class

In [ ]:

# Setting string representing path for folder containing audio files
Ravdess = '/Users/stephen/Emotion_Detection-/data/RAVDESS/'

# Turn Path string into Path Object 
ravdess_directory_list = os.listdir(Ravdess)
ravdess_directory_list.sort

# Create empty lists to store audio paths and their labels associated with each file 
file_emotion = []
file_path = []
# Since each actor is in there own folder we must iterate through each folder 
for dir in ravdess_directory_list:
    # as their are 24 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        # splitting up file name to decode labels 
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

In [ ]:
Ravdess_df

In [ ]:
# Creating dataset object 

class Dataset(Dataset):
    def __init__(self, annotations_file, audio_dir, transform=None):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        self.transform = transform 

    # Get the length 
    def __init__(self):
        return len(self.annotations)

    # get items 
    def __getitem__(self, idx):
        # identify path
        path = self.annotations(idx)
        # open wav file with torchaudio and returns tensor 
        signal, sample_rate = torchaudio.load(path)
        # get label 
        label = self.annotations[idx, 4]
        # if there is any transform method, apply to signal 
        if self.transform:
            signal = self.transform(signal)
        return signal, label 

if __name__ == "__main__":
    annotations_file = "/Users/stephen/Emotion_Detection-/data/metadata.csv"
    audio_dir = "/Users/stephen/Emotion_Detection-/data/RAVDESS"

    speech = Dataset(annotations_file, audio_dir)

    
        





In [ ]:
data = Dataset()

In [ ]:
data[1]

In [89]:
class UrbanSoundDataset(Dataset):

    def __init__(self,
                 annotations_file,
                 audio_dir,
                 transformation,
                 target_sample_rate,
                 num_samples,
                 device):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        self.device = device
        #self.transformation = transformation.to(self.device)
        self.target_sample_rate = target_sample_rate
        self.num_samples = num_samples

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device)
        signal = self._resample_if_necessary(signal, sr)
        signal = self._mix_down_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        signal = self._right_pad_if_necessary(signal)
        signal = self.transformation(signal)
        return signal, label

    def _cut_if_necessary(self, signal):
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        return signal

    def _right_pad_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

    def _resample_if_necessary(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        return signal

    def _mix_down_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

    def _get_audio_sample_path(self, index):
        fold = f"fold{self.annotations.iloc[index, 5]}"
        path = os.path.join(self.audio_dir, fold, self.annotations.iloc[
            index, 0])
        return path

    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 6]


if __name__ == "__main__":
    ANNOTATIONS_FILE = "/Users/stephen/Emotion_Detection-/data/metadata.csv"
    AUDIO_DIR = "/Users/stephen/Emotion_Detection-/data/RAVDESS"
    SAMPLE_RATE = 22050
    NUM_SAMPLES = 22050

    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using device {device}")

    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )

    usd = UrbanSoundDataset(ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            None,
                            SAMPLE_RATE,
                            NUM_SAMPLES,
                            device)
    print(f"There are {len(usd)} samples in the dataset.")
    signal, label = usd[0]

Using device cpu
There are 1380 samples in the dataset.


formats: can't open input file `/Users/stephen/Emotion_Speech_Detection/data/RAVDESS/Actor_01/03-01-08-02-02-01-01.wav': No such file or directory


RuntimeError: Error loading audio file: failed to open file /Users/stephen/Emotion_Speech_Detection/data/RAVDESS/Actor_01/03-01-08-02-02-01-01.wav

In [ ]:
/Users/stephen/Emotion_Detection-/data/RAVDESS/Actor_01/03-01-08-02-02-01-01.wav